# Question-answering engine Part 3


In [ ]:
!pip install datasets transformers beir

## Compress all into functions for easier use

In [3]:
def prepare_train_features(examples, tokenizer):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    max_length = 384 # The maximum length of a feature (question and context)
    doc_stride = 128 # The authorized overlap between two part of the context when splitting it is needed.
    pad_on_right = tokenizer.padding_side == "right"

    # Tokenize our examples with truncation and padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # The offset mappings will give us a map from token to character position in the original context. This will
    # help us compute the start_positions and end_positions.
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # Let's label those examples!
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        # If no answers are given, set the cls_index as answer.
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                token_end_index -= 1

            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                # Note: we could go after the last offset if the answer is the last word (edge case).
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

In [4]:
def prepare_validation_features(examples, tokenizer):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    max_length = 384 # The maximum length of a feature (question and context)
    doc_stride = 128 # The authorized overlap between two part of the context when splitting it is needed.
    pad_on_right = tokenizer.padding_side == "right"
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and maybe padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

    # We keep the example_id that gave us this feature and we will store the offset mappings.
    tokenized_examples["example_id"] = []

    for i in range(len(tokenized_examples["input_ids"])):
        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1 if pad_on_right else 0

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["id"][sample_index])

        # Set to None the offset_mapping that are not part of the context so it's easy to determine if a token
        # position is part of the context or not.
        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]

    return tokenized_examples

In [5]:
def postprocess_qa_predictions(examples, tokenizer, features, raw_predictions, n_best_size = 20, max_answer_length = 30):
    all_start_logits, all_end_logits = raw_predictions
    # Build a map example to its corresponding features.
    example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)

    # The dictionaries we have to fill.
    predictions = collections.OrderedDict()

    # Logging.
    print(f"Post-processing {len(examples)} example predictions split into {len(features)} features.")

    # Let's loop over all the examples!
    for example_index, example in enumerate(tqdm(examples)):
        # Those are the indices of the features associated to the current example.
        feature_indices = features_per_example[example_index]

        min_null_score = None # Only used if squad_v2 is True.
        valid_answers = []
        
        context = example["context"]
        # Looping through all the features associated to the current example.
        for feature_index in feature_indices:
            # We grab the predictions of the model for this feature.
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]
            # This is what will allow us to map some the positions in our logits to span of texts in the original
            # context.
            offset_mapping = features[feature_index]["offset_mapping"]

            # Update minimum null prediction.
            cls_index = features[feature_index]["input_ids"].index(tokenizer.cls_token_id)
            feature_null_score = start_logits[cls_index] + end_logits[cls_index]
            if min_null_score is None or min_null_score < feature_null_score:
                min_null_score = feature_null_score

            # Go through all possibilities for the `n_best_size` greater start and end logits.
            start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
            end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
                    # to part of the input_ids that are not in the context.
                    if (
                        start_index >= len(offset_mapping)
                        or end_index >= len(offset_mapping)
                        or offset_mapping[start_index] is None
                        or offset_mapping[end_index] is None
                    ):
                        continue
                    # Don't consider answers with a length that is either < 0 or > max_answer_length.
                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                        continue

                    start_char = offset_mapping[start_index][0]
                    end_char = offset_mapping[end_index][1]
                    valid_answers.append(
                        {
                            "score": start_logits[start_index] + end_logits[end_index],
                            "text": context[start_char: end_char]
                        }
                    )
        
        if len(valid_answers) > 0:
            best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
        else:
            # In the very rare edge case we have not a single non-null prediction, we create a fake prediction to avoid
            # failure.
            best_answer = {"text": "", "score": 0.0}
        
        # Let's pick our final answer: the best one or the null answer (only for squad_v2)
        answer = best_answer["text"] if best_answer["score"] > min_null_score else ""
        predictions[example["id"]] = answer
        
    return predictions

In [6]:
def fine_tune_model(checkpoint, datasets, saved_file=None, train=True):
  """
  Prepare dataset and fine-tune a model using pre-trained values (checkpoint).

  Input:
    * Checkpoint: Checkpoint of a Question_Answering model from the Huggingface library.
    * datasets: Dict of key "train" and "validation" and value two datasets.
    * save_file: Path to model save file

  Return:
    A fine-tuned trainer object.
  """

  # Hyperparameters
  batch_size = 16

  # Create first tokenizer
  tokenizer = AutoTokenizer.from_pretrained(checkpoint)
  
  # Tokenize datasets
  #train_ds = datasets['train'].map(lambda x: prepare_train_features(x, tokenizer), batched=True, remove_columns=datasets['train'].column_names)
  #val_ds = datasets['validation'].map(lambda x: prepare_train_features(x, tokenizer), batched=True, remove_columns=datasets['train'].column_names)

  # Create model
  if saved_file is None:
    model = AutoModelForQuestionAnswering.from_pretrained(checkpoint)
  else:
    model = AutoModelForQuestionAnswering.from_pretrained(saved_file, local_files_only=True)
    model.to('cuda')

  # Model parameters
  model_name = checkpoint.split("/")[-1]
  args = TrainingArguments(
    f"{model_name}-finetuned-squad",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
  )

  # Create datacollator
  data_collator = default_data_collator

  # Create trainer
  trainer = Trainer(
    model,
    args,
    data_collator=data_collator,
    tokenizer=tokenizer,
  )

  # Train model
  if train:
    trainer.train()

  # Save model
  if train:
    trainer.save_model(model_name + "_save")

  return trainer, tokenizer

In [101]:
def evaluate_model(sample, trainer, tokenizer) -> list:
  # Hyperparameters
  n_best_size = 20
  max_answer_length = 30
  
  # Prepare validation datasets
  validation_features = sample.map(
    lambda x : prepare_validation_features(x, tokenizer),
    batched=True,
    remove_columns=datasets["validation"].column_names
  )

  # Calculate raw predictions
  raw_predictions = trainer.predict(validation_features)

  # Add usefull columns
  validation_features.set_format(type=validation_features.format["type"], columns=list(validation_features.features.keys()))

  # Retrieve features
  examples = sample
  features = validation_features

  example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
  features_per_example = collections.defaultdict(list)
  for i, feature in enumerate(features):
      features_per_example[example_id_to_index[feature["example_id"]]].append(i)

   #Calculate final predictions
  final_predictions = postprocess_qa_predictions(sample, tokenizer, validation_features, raw_predictions.predictions)

  # Get metric for evaluation
  #metric = load_metric("squad_v2")

  # Evaluate model
  #formatted_predictions = [{"id": k, "prediction_text": v, "no_answer_probability": 0.0} for k, v in final_predictions.items()]
  formatted_predictions = [v  for k, v in final_predictions.items() if v != ""]
  
  #references = [{"id": ex["id"], "answers": ex["answers"]} for ex in datasets["validation"]]
  return formatted_predictions

# Full code

In [8]:
# Import
import transformers
import collections
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
from beir import util
from beir.datasets.data_loader import GenericDataLoader
import sentence_transformers
from sentence_transformers import SentenceTransformer
import torch
import random
from transformers import AutoTokenizer
from datasets import load_dataset, load_metric
from transformers import default_data_collator
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer

## Create dataset from SQUADV2 and Dbpedia

In [9]:
# Get model checkpoint
teacher_checkpoint = "mvonwyl/distilbert-base-uncased-finetuned-squad2"

# Get datasets
datasets_ = load_dataset("squad_v2", split=['train[:1]', 'validation[:5000]'])
dataset = "dbpedia-entity"
url = "https://public.ukp.informatik.tu-darmstadt.de/thakur/BEIR/datasets/{}.zip".format(dataset)
data_path = util.download_and_unzip(url, "datasets")
corpus, queries, qrels = GenericDataLoader(data_folder=data_path).load(split="test")


# Create dataset dictionnary
datasets = {}
datasets["train"] = datasets_[0]
datasets["validation"] = datasets_[1]
corpus_items_list = list(corpus.items())
shuffled_sample = random.sample(corpus_items_list, 5000)
for _, c in shuffled_sample:
    title = c['title']
    context = c['text']
    d = {'id' : '', 'title': title, 'context': context, 'question': '', 'answers': None, 'answer_start': None}
    datasets["validation"] = datasets["validation"].add_item(d)

Downloading:   0%|          | 0.00/1.87k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.02k [00:00<?, ?B/s]

  0%|          | 0/2 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/9.55M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/801k [00:00<?, ?B/s]

...

0 examples [00:00, ? examples/s]

Dataset squad_v2 downloaded and prepared to /root/.cache/huggingface/datasets/squad_v2/squad_v2/2.0.0/09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

datasets/dbpedia-entity.zip:   0%|          | 0.00/610M [00:00<?, ?iB/s]

  0%|          | 0/4635922 [00:00<?, ?it/s]

In [10]:
# Get best vectorizer model
model_distilbert = SentenceTransformer('msmarco-distilbert-cos-v5')

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/5.12k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/545 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/349 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/319 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [11]:
# Calculate all vector
df = pd.DataFrame({'title' : [], 'context': [], 'question':[], 'encode-distilbert': []})
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

for i in range(len(datasets['validation'])):
    dt_context = datasets['validation']["context"][i]
    dt_title = datasets['validation']["title"][i]
    dt_question = datasets['validation']["question"][i]
    di = {"title": dt_title, "context" : dt_context, "question": dt_question, 
          "encode-distilbert": model_distilbert.encode(dt_context,device=device)}
    df = df.append(di, ignore_index=True)

In [93]:
# Calculate cosine similarity
def cos_sim(a: np.ndarray, b: np.ndarray):
  """
  Calculate the cosine similarity betweern a and b
  """
  if a is None or b is None:
    return None
  return sentence_transformers.util.pytorch_cos_sim(a, b)

def calculate_best_similarity_matrice(question: str) -> np.ndarray:
    """
    Calculate the similarity matrix
    """
    # Get encoded question
    question_matrix_dist = model_distilbert.encode(question,device=device)

    # Get matrix of all paragraph
    c_vec_list = []
    for c_vec in df[f"encode-distilbert"]:
        if c_vec is not None:
            c_vec_list.append(c_vec)

    context_matrix_dist = np.vstack(c_vec_list)
    cos_matrix_dist = cos_sim(question_matrix_dist, context_matrix_dist)
    argsort_dist = torch.argsort(cos_matrix_dist, dim=1)

    return argsort_dist

def get_n_best_matches(question: str, n: int = 5) -> (pd.DataFrame, int):
  """
  Get n most similar context to the question asked
  """
  best_matches_indexes = calculate_best_similarity_matrice(question)[0]

  i = 1
  num_added = 0
  len_df = len(df)
  res = pd.DataFrame(columns=df.columns)
  while num_added != n and i <= len_df:
    if df.iloc[int(best_matches_indexes[-i])]["context"] not in res["context"].to_list():
      res = res.append(df.iloc[int(best_matches_indexes[-i])])
      num_added += 1

    i += 1

  return res

## Answering our question

In [35]:
# Create Model
teacher_trainer, teacher_tokenizer = fine_tune_model(teacher_checkpoint, datasets, train=False)

Downloading:   0%|          | 0.00/333 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/561 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/253M [00:00<?, ?B/s]

In [89]:
def answer_question(question: str, trainer, tokenizer) -> list:
  """
  Retrieve some answers for a given question

  Input:
  question: str
  trainer: trainer
  tokenizer: tokenizer
  """
  best_contexts = get_n_best_matches(question, 10)
  data = datasets_[0]

  samples = []

  for ii, context in enumerate(best_contexts['context']): 
    samples.append({"title":"", "id":str(ii), "context":context, "question":question, "answers":None, 'answer_start': None})

  for c in samples:
    data = data.add_item(c)

  return evaluate_model(data, trainer, tokenizer)[1:]

In [94]:
answers = answer_question("Who ruled France ?", teacher_trainer, teacher_tokenizer)

Loading cached processed dataset at /root/.cache/huggingface/datasets/squad_v2/squad_v2/2.0.0/09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d/cache-7dac3297b954992b.arrow
The following columns in the test set  don't have a corresponding argument in `DistilBertForQuestionAnswering.forward` and have been ignored: example_id, offset_mapping.
***** Running Prediction *****
  Num examples = 11
  Batch size = 16


Post-processing 11 example predictions split into 11 features.


  0%|          | 0/11 [00:00<?, ?it/s]

In [95]:
answers

["Louis Philippe d'Orléans",
 'Claude François Chauveau-Lagarde',
 'Henry of Navarre',
 'Pierre de Langle',
 'French Crown',
 'grand audiencier of France']

When asking "Who ruled France ?". We get answers that are great such as 'Henry of Navarre' or 'Louis Philippe d'Orléans' but also some mistakes. Nevertheless the model seems to grasp nicely the concept of France and ruling.  